In [ ]:

!pip install geemap


In [ ]:

import ee

# Initialise la session après authentification OAuth2 Google
ee.Initialize(project='lyra-teledetection')

# Coordonnées du site Haute-Loire (décimal)
point = ee.Geometry.Point([4.0966666, 45.0430555])

# ROI pour le calcul du NDVI : très local (10 m)
ndvi_roi = point.buffer(10)

# ROI pour affichage du NDVI : plus grand, par exemple 1000 m pour un carré plus lisible
ndvi_display_roi = point.buffer(1000).bounds()  # bounds() pour carré

# ROI pour le calcul de la LST : plus large (5 km)
lst_roi = point.buffer(5000)

# NDVI Sentinel-2 (15 mai – 15 juin 2025)
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \    .filterDate('2025-05-15', '2025-06-15') \    .filterBounds(point) \    .sort('CLOUDY_PIXEL_PERCENTAGE') \    .first()

ndvi = s2.normalizedDifference(['B8', 'B4']).rename('NDVI')

# Clipper le NDVI pour l'affichage (évite un grand carré par défaut)
ndvi_clipped = ndvi.clip(ndvi_display_roi)

mean_ndvi = ndvi.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=ndvi_roi,
    scale=10,
    maxPixels=1e9
)

# LST MODIS (15 mai – 15 juin 2025)
lst_collection = ee.ImageCollection('MODIS/061/MOD11A1') \    .filterDate('2025-05-15', '2025-06-15') \    .filterBounds(lst_roi)

lst_image = lst_collection.mean().select('LST_Day_1km').multiply(0.02).subtract(273.15).rename('LST_Celsius')

mean_lst = lst_image.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=lst_roi,
    scale=1000,
    maxPixels=1e9
)

# Fusion propre, NDVI à 2 décimales, LST à 1 décimale
combined = {
    'NDVI': round(mean_ndvi.getInfo().get('NDVI'), 2),
    'LST_Celsius': round(mean_lst.getInfo().get('LST_Celsius'), 1)
}

print('NDVI + LST combinés (arrondis) :', combined)


In [ ]:

import geemap.foliumap as geemap

# Créer la carte Folium centrée sur le point Haute-Loire
Map = geemap.Map(center=[45.0430555, 4.0966666], zoom=12)

Map.add_basemap('HYBRID')

# NDVI VISUALISATION — version clippée pour un carré plus resserré
ndvi_vis = ndvi_clipped.visualize(min=0, max=1, palette=['white', 'green'])

# Ajouter le NDVI limité, le point et les buffers
Map.addLayer(ndvi_vis, {}, 'NDVI (clippé)')
Map.addLayer(point, {'color': 'red'}, 'Site Haute-Loire')
Map.addLayer(ndvi_display_roi, {'color': 'green'}, 'ROI NDVI affiché (carré 1 km)')
Map.addLayer(lst_roi, {'color': 'blue'}, 'Buffer LST (5 km)')

Map
